In [ ]:
%%capture
!pip install langchain-openai
!pip install langchain
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade
!pip install bert_score


In [ ]:
import nltk
from datasets import load_dataset, Dataset, Features, Value, ClassLabel
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

##Reading the data
Using all the functions from previous script to read the data.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, json
import numpy as np
from argparse import ArgumentParser
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from pprint import pprint
from datetime import datetime
import copy

In [ ]:
#@title Reading train, dev and test data
folder_path = 'path'

def read_json_file(name):
    with open(name, 'r') as f:
        data = json.load(f)
        return data

def read_data():
    train = read_json_file(os.path.join(folder_path, "train.json"))
    dev = read_json_file(os.path.join(folder_path, "dev.json"))
    test = read_json_file(os.path.join(folder_path, "test.json"))

    return train, dev, test

train, dev, test = read_data()
print(len(train), len(dev), len(test))

246 50 100


## Preparing the Question and Answer pair

In [ ]:
def read_json_file(name):
    folder_path = 'Final-Data'
    name = os.path.join(folder_path, name)
    with open(name, 'r') as f:
        data = json.load(f)
        return data

def save_json(json_data, file_name):
    json_data = json.dumps(json_data)
    print(file_name)
    with open(file_name, "w") as json_file:
          json_file.write(json_data)

role_details = read_json_file(f'role_definitions_question_guided.json')
pprint(role_details)
instruction_prefix = '''Concisely extract the following argument from the post comment pair.
Do not use more than 12 words to describe an argument. Return 'null' if any arugment is not present.'''

{'relapse': {'description': '',
             'event-specific-arguments': {'condition': 'What is the substance '
                                                       'use history/disorder '
                                                       'from which the subject '
                                                       'had previously '
                                                       'recovered or was in '
                                                       'the process of '
                                                       'recovering?',
                                          'existing/current-medications': 'Which '
                                                                          'medication '
                                                                          'did '
                                                                          'the '
                                                                          'subject '
                

In [ ]:
#creating the QA pair with the instructions.

def create_QA_pair(data, role_details):
    arg_types = ['main-arguments', 'event-specific-arguments', 'subject-effect-arguments']
    qa_dt = []
    i=0
    for dt in data:
        post, comment, label = dt['text1'], dt['text2'], dt['label']
        context = f'''Post: {post}\nComment: {comment}''' ## adding post and comment togethar to create context

        for arg_typ in arg_types:
            role_descriptions = role_details[label][arg_typ] #chosing the argument details from the correspond event
            arguments = list(role_descriptions.keys()) ##getting specific arguments

            for arrg in arguments:
                arg_question = role_details[label][arg_typ][arrg]
                answer = dt['ground-truth-arguments'][arg_typ][arrg]
                input_dict = {
                  'id' : f'ins_{i}',
                  'instruction' : instruction_prefix,
                  'context' : context,
                  'question' : arg_question,
                  'answer' : ", ".join(answer)
                }
                i+=1
                qa_dt.append(input_dict)
    return qa_dt


In [ ]:
from transformers import pipeline

model_checkpoint = "huggingface-course/bert-finetuned-squad"
question_answerer = pipeline("question-answering", model=model_checkpoint)

In [ ]:
def create_predictions(data, role_details, decoded_predictions):
    i = 0
    arg_types = ['main-arguments', 'event-specific-arguments', 'subject-effect-arguments']
    finallist = []
    for dt in data:
        pprint(dt)
        dx = dt
        label = dt['label']
        predictions = {}
        for arg_typ in arg_types:
            role_descriptions = role_details[label][arg_typ] #chosing the argument details from the correspond event
            arguments = list(role_descriptions.keys()) ##getting specific arguments
            predictions[arg_typ] = {}
            for arrg in arguments:
                print(arg_typ, arrg)
                print(decoded_predictions[i])
                value = decoded_predictions[i]
                value_list = [item.replace("'", "").strip() for item in value.split(',')]
                predictions[arg_typ][arrg] = value_list
                i+=1
        dx['predictions'] = predictions
        pprint(dx)
        finallist.append(dx)
    return finallist

In [ ]:
def save_json(json_data, file_name):
    json_data = json.dumps(json_data)
    print(file_name)
    with open(file_name, "w") as json_file:
          json_file.write(json_data)


In [ ]:
%%time
numbers = ['0', '1', '2']
for number in numbers:
    print(number)
    predictions = []
    for qa in qa_test:
        question = qa['question']
        context = qa['context']
        # print(question)
        # print(context)
        result = question_answerer(question=question, context=context)
        print(result)
        predictions.append(result['answer'])
    final_data = create_predictions(test, role_details, predictions)
    folder_path= 'Argument-Extraction-Predictions'
    file_name = f'{number}-extractive_qa_fine_tuned_squad-predictions.json'
    save_json(final_data, os.path.join(folder_path, file_name))